In [24]:
from langchain.output_parsers.json import SimpleJsonOutputParser
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage 
from dotenv import load_dotenv
from datetime import datetime
from os.path import exists
import tiktoken
import re
import os
import instructor
from openai import OpenAI
from pydantic import BaseModel, Field
from typing import List, Optional, Literal
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from langchain.output_parsers.json import SimpleJsonOutputParser
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain.schema import StrOutputParser
import json
from tqdm import tqdm


In [13]:
def read_jsonl_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return [json.loads(line) for line in file if line.strip()]
    

def write_jsonl_file(file_path, data):
    # Open the file in write mode ('w')
    with open(file_path, 'w') as file:
        for item in data:
            # Serialize the dictionary to a JSON formatted string
            json_str = json.dumps(item)
            # Write the JSON string to the file followed by a newline
            file.write(json_str + '\n')


In [73]:
data = read_jsonl_file('final_target_scores.jsonl')

In [59]:
jewelry = [product for product in data if product['category'] == 'Jewelry']
len(jewelry)

250

In [53]:
office = [product for product in data if product['category'] == 'Office Products']
len(office)

297

In [70]:
food = [product for product in data if product['category'] == 'Grocery And Gourmet Food']
len(food)

81

In [74]:
home_garden = [product for product in data if product['category'] == 'Home And Garden']

In [36]:
food[0]

{'id': 274490,
 'cluster_id': 141824.0,
 'category': 'Grocery And Gourmet Food',
 'input_title': 'Nutiva Organic Hemp Protein Hi-Fiber -- 3 lbs',
 'input_description': "Per Serving 12 g Fiber 11 g Protein Nutiva's delicious Organic Hemp Protein Powder is: Extremely nutritious. One serving provides 11 grams raw organic protein and a whopping 14 grams of fiber (54% of the RDI). Hemp contains all 8 essential amino acids with the bonus of good-for-you essential fatty acids. Certified Organic. No GMOs, pesticides, or solvents! Ever wonder why soy burgers and soymilk are organic yet soy protein isn't? Most non-organic soy protein is processed with hexane, a petroleum solvent similar to gasoline. Planet-friendly. Hemp (a non-drug crop) is the earth's premier renewable resource and grows easily without pesticides.",
 'target_scores': {'Brand': {'Nutiva': {'pid': [0, 1], 'score': 1}},
  'Product Type': {'Hemp Protein': {'pid': [0], 'score': 1},
   'Hemp Protein Powder': {'pid': [1], 'score': 1}

In [62]:
key_set = {}

attributes_list = []
for i in jewelry:
   attributes_list += list(i['target_scores'].keys())

attributes = set(attributes_list)
attributes

{'Brand',
 'Gender',
 'Metal Type',
 'Model Number',
 'Product Type',
 'Stone Shape',
 'Stones Setting',
 'Stones Total Weight',
 'Stones Type'}

In [63]:
from pydantic import BaseModel
from typing import Optional

class ProductInfoGarden(BaseModel):
   Base: Optional[str] = Field(default=None, alias='Base')
   Capacity: Optional[str] = Field(default=None, alias='Capacity')
   Color: Optional[str] = Field(default=None, alias='Color')
   Cooling: Optional[str] = Field(default=None, alias='Cooling')
   Depth: Optional[str] = Field(default=None, alias='Depth')
   Gauge: Optional[str] = Field(default=None, alias='Gauge')
   Heat: Optional[str] = Field(default=None, alias='Heat')
   Height: Optional[str] = Field(default=None, alias='Height')
   Length: Optional[str] = Field(default=None, alias='Length')
   Manufacturer: Optional[str] = Field(default=None, alias='Manufacturer')
   Manufacturer_Stock_Number: Optional[str] = Field(default=None, alias='Manufacturer Stock Number')
   Material: Optional[str] = Field(default=None, alias='Material')
   Product_Type: Optional[str] = Field(default=None, alias='Product Type')
   Retail_UPC: Optional[str] = Field(default=None, alias='Retail UPC')
   Shape: Optional[str] = Field(default=None, alias='Shape')
   Shelves: Optional[str] = Field(default=None, alias='Shelves')
   Splash: Optional[str] = Field(default=None, alias='Splash')
   Stainless_Steel_Series: Optional[str] = Field(default=None, alias='Stainless Steel Series')
   Voltage: Optional[str] = Field(default=None, alias='Voltage')
   Width: Optional[str] = Field(default=None, alias='Width')


class ProductInfoFood(BaseModel):

   Brand: Optional[str] = Field(default=None, alias='Brand')
   Flavor: Optional[str] = Field(default=None, alias='Flavor')
   Manufacturer_Stock_Number: Optional[str] = Field(default=None, alias='Manufacturer Stock Number')
   Pack_Quantity: Optional[str] = Field(default=None, alias='Pack Quantity')
   Packing_Type: Optional[str] = Field(default=None, alias='Packing Type')
   Product_Type: Optional[str] = Field(default=None, alias='Product Type')
   Retail_UPC: Optional[str] = Field(default=None, alias='Retail UPC')
   Size_Weight: Optional[str] = Field(default=None, alias='Size/Weight')
   
   
class ProductInfoOffice(BaseModel):
   Brand: Optional[str] = Field(default=None, alias='Brand')
   Binding: Optional[str] = Field(default=None, alias='Binding')
   Capacity: Optional[str] = Field(default=None, alias='Capacity')
   Closure: Optional[str] = Field(default=None, alias='Closure')
   Colors: Optional[str] = Field(default=None, alias='Color(s)')
   Depth: Optional[str] = Field(default=None, alias='Depth')
   Height: Optional[str] = Field(default=None, alias='Height')
   Length: Optional[str] = Field(default=None, alias='Length')
   Width: Optional[str] = Field(default=None, alias='Width')
   Mounting: Optional[str] = Field(default=None, alias='Mounting')
   Manufacturer_Stock_Number: Optional[str] = Field(default=None, alias='Manufacturer Stock Number')
   Material: Optional[str] = Field(default=None, alias='Material')
   Product_Type: Optional[str] = Field(default=None, alias='Product Type')
   Retail_UPC: Optional[str] = Field(default=None, alias='Retail UPC')
   Page_Yield: Optional[str] = Field(default=None, alias='Page Yield')
   Paper_Weight: Optional[str] = Field(default=None, alias='Paper Weight')
   Sheet_Perforation: Optional[str] = Field(default=None, alias='Sheet Perforation')
   Pack_Quantity: Optional[str] = Field(default=None, alias='Pack_Quantity')
   
class ProductInfoJewelry(BaseModel):
   Brand: Optional[str] = Field(default=None, alias='Brand')
   Gender: Optional[str] = Field(default=None, alias='Gender')
   Metal_Type: Optional[str] = Field(default=None, alias='Metal Type')
   Model_Number: Optional[str] = Field(default=None, alias='Model Number')
   Product_Type: Optional[str] = Field(default=None, alias='Product Type')
   Stone_Shape: Optional[str] = Field(default=None, alias='Stone Shape')
   Stones_Setting: Optional[str] = Field(default=None, alias='Stones Setting')
   Stones_Total_Weight: Optional[str] = Field(default=None, alias='Stones Total Weight')
   Stones_Type: Optional[str] = Field(default=None, alias='Stones Type')


 


In [75]:
def initial_extraction(text: str, custom_extraction_prompt:str, model_name: str = 'gpt-4o') -> ProductInfoGarden:
    
    # Patch the OpenAI client with Instructor
    client = instructor.from_openai(OpenAI(api_key=os.getenv('OPENAI_KEY')))
    
    system_message = """
    You are an intelligent text extraction and conversion assistant. Your task is to extract structured information 
    from the given text and convert it into a structured format. 
    The output response should contain only the data extracted from the text, with no additional commentary, explanations, or extraneous information.
    If the required information could not be found from the given source, return nothing for that field. Do not hallucinate.
    """
    
    rule_prompt = """
                Here are the rules that you need to adhere:
                    ## Rules:
                    - The aim is to achieve simplicity and clarity in the extracted text.
                    - Make sure to answer in the structured format.
                    - If no information is provided for any of the fields, return nothing of that field.
                    - DO NOT HALLUCINATE.
                """
    
    extraction_prompt = f"""
    {system_message}
    {custom_extraction_prompt}
    """

    response = client.chat.completions.create(
        model=model_name, 
        response_model=ProductInfoGarden,
        messages=[
            {"role": "system", "content": extraction_prompt},
            {"role": "user", "content": f"Use the given text to extract information: {text}"},
            {"role": "user", "content": rule_prompt}
        ]
    )
    return response

def evaluate_extraction(my_output, expected_output):
    
    # Simplify the expected output by removing the details of pid and score and handling multiple valid options
    simplified_expected_output = {'target_scores': {}}

    for field, values in expected_output['target_scores'].items():
        if isinstance(values, dict):  # Check if the value is a dictionary (implying multiple values)
            # Extract keys and make them a list if there are multiple valid responses
            simplified_expected_output['target_scores'][field] = list(values.keys())
        else:
            # Directly assign if it is a single value
            simplified_expected_output['target_scores'][field] = values
    
    target_scores = simplified_expected_output['target_scores']
    total_fields = len(target_scores)  # Total fields to be extracted
    correct_matches = 0  # Count of correctly extracted fields
    incorrect_fields = []  # List to store names of fields that are incorrect

    for field, expected_values in target_scores.items():
        # Normalize the value from my_output
        field = field.replace(' ', '_')
        field = field.replace('/', '_')
        field = field.replace('(', '')
        field = field.replace(')', '')
        my_value = my_output.get(field, 'n/a') if my_output.get(field) is not None else 'n/a'
        my_value = my_value.lower()  # Convert to lower case for case insensitive comparison

        # Handle cases where multiple correct answers are expected
        if isinstance(expected_values, list):
            # Normalize all expected values for case-insensitive comparison
            expected_values_normalized = [value.lower() for value in expected_values]
            if my_value in expected_values_normalized:
                correct_matches += 1
            else:
                incorrect_fields.append(field)
        else:  # Single correct answer case
            expected_value_normalized = expected_values.lower()
            if my_value == expected_value_normalized:
                correct_matches += 1
            else:
                incorrect_fields.append(field)

    accuracy_percentage = (correct_matches / total_fields) * 100

    # Print fields that were incorrect
    if incorrect_fields:
        print("Incorrect fields:")
        for field in incorrect_fields:
            print(f"- {field}")

    return accuracy_percentage




In [78]:
result_list = []
accuracy_list = []
model_name = 'gpt-4o-mini' #'gpt-3.5-turbo'
dataset = 'garden'

for index in tqdm(range(0, 100)):
    result = {}
    data = home_garden[index]
    result['id'] = data['id']
    result['cluster_id'] = data['cluster_id']
    result['category'] = data['category']
    text_compile = f'title: {data["input_title"]}\ndescription: {data["input_description"]}'

    custom_extraction_prompt = """
        Extract information from the text extracted from a webpage of a product, including title and description:
        Output in a structured format.
        """
    response = initial_extraction(text_compile, custom_extraction_prompt, model_name)
    result['output'] = response.dict()
    accuracy = evaluate_extraction(response.dict(), data)
    result['accuracy'] = accuracy
    result_list.append(result)
    accuracy_list.append(accuracy)
print(f'Average accuracy: {sum(accuracy_list)/len(accuracy_list)}')

# Specify the file path for your JSON Lines file
file_path = f'{model_name.replace(".", "-")}-{dataset}-zero.jsonl'

write_jsonl_file(file_path, result_list)



  1%|          | 1/100 [00:02<04:28,  2.71s/it]

Incorrect fields:
- Width
- Length
- Depth
- Gauge
- Material
- Stainless_Steel_Series
- Height
- Base


  2%|▏         | 2/100 [00:05<04:29,  2.75s/it]

Incorrect fields:
- Manufacturer
- Product_Type
- Width
- Depth
- Gauge
- Material
- Stainless_Steel_Series
- Height
- Manufacturer_Stock_Number


  3%|▎         | 3/100 [00:07<04:13,  2.62s/it]

...

Incorrect fields:
- Manufacturer
- Base


 99%|█████████▉| 99/100 [03:56<00:02,  2.40s/it]

Incorrect fields:
- Manufacturer_Stock_Number
- Base


100%|██████████| 100/100 [03:59<00:00,  2.39s/it]

Incorrect fields:
- Height
- Width
- Length
- Depth
- Stainless_Steel_Series
- Cooling
- Splash
- Shape
- Retail_UPC
- Manufacturer_Stock_Number
- Heat
- Gauge
- Base
- Voltage
- Capacity
Average accuracy: 61.4


In [83]:
len(home_garden)

356

- garden - gpt3.5 - zero - 75.65
- garden - gpt4o - zero - 81.2
- garden - gpt4o-mini - zero - 62.3, 61.4

- food - gpt3.5 - zero - 70.21604938271605
- food - gpt4o - zero - 77.62345679012346, 75.92592592592592
- food - gpt4o-mini - zero - 74.07407407407408

- office - gpt3.5 - zero - 76.9444444444444
- office - gpt4o - zero - 78.11111111111109
- office - gpt4o-mini - zero - 76.05555555555554

- jewelry - gpt3.5 - zero - 76.77777777777769
- jewelry - gpt4o - zero - 84.55555555555544
- jewelry - gpt4o-mini - zero - 82.3333333333332





In [ ]:
Example = """
        Here is an example of the information extraction tasks
        
        ## Input Text
        title: John Boos EBMS6-2024 24" Solid Wall Mounted Shelving w/ Raised Ledges Ledges
        description: Microwave Shelf, wall mount, 24"W x 20"D x 15"H, 16 gauge 300 series stainless steel, Stallion safety edge front with raised lip, NSF
        
        ## Expected Output
        'Manufacturer': 'John Boos',
        'Product Type': 'Wall Mounted Shelving'
        'Width': '24"'
        'Length': None
        'Depth': '20"'
        'Gauge': '16'
        'Material': 'stainless steel'
        'Stainless Steel Series': '300'
        'Cooling': None 
        'Splash': None
        'Shape': None
        'Height': '15"'
        'Color': None
        'Retail UPC': None
        'Manufacturer Stock Number': 'EBMS6-2024'
        'Heat': None
        'Shelves': None
        'Base': None
        'Voltage': None
        'Capacity': None
    """

In [18]:
a = [1,2,3]


2.0